In [ ]:
import os
import urllib.request
from pathlib import Path

# Создаем папку для датасета
dataset_dir = Path('/content/imdb_wiki')
dataset_dir.mkdir(exist_ok=True)

print(f"Папка для датасета создана: {dataset_dir}")

Папка для датасета создана: /content/imdb_wiki


In [ ]:
# URL для загрузки метаданных IMDB-WIKI
imdb_meta_url = "https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_meta.tar"
wiki_meta_url = "https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_meta.tar"

# Функция для загрузки с прогрессом
def download_with_progress(url, filename):
    def progress_hook(block_num, block_size, total_size):
        downloaded = block_num * block_size
        if total_size > 0:
            percent = min(100, (downloaded * 100) // total_size)
            print(f"\rЗагружено: {percent}%", end='')

    print(f"Загружаем {filename}...")
    urllib.request.urlretrieve(url, filename, progress_hook)
    print(f"\n {filename} загружен!")

# Загружаем метаданные
download_with_progress(imdb_meta_url, dataset_dir / "imdb_meta.tar")

Загружаем /content/imdb_wiki/imdb_meta.tar...
Загружено: 100%
✅ /content/imdb_wiki/imdb_meta.tar загружен!


In [ ]:
import tarfile
import scipy.io
import numpy as np

# Извлекаем архив с метаданными
with tarfile.open(dataset_dir / "imdb_meta.tar", 'r') as tar:
    tar.extractall(dataset_dir)
    print(" Метаданные извлечены!")

# Загружаем .mat файл с информацией о датасете
mat_file = dataset_dir / "imdb_meta.mat"
if mat_file.exists():
    print("Найден файл с метаданными")
else:
    # Поищем файл в извлеченных папках
    mat_files = list(dataset_dir.rglob("*.mat"))
    if mat_files:
        mat_file = mat_files[0]
        print(f"Найден файл: {mat_file}")
    else:
        print(" .mat файл не найден")

✅ Метаданные извлечены!
📁 Найден файл: /content/imdb_wiki/imdb/imdb.mat


/tmp/ipython-input-3867926990.py:7: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(dataset_dir)


In [ ]:
# Загружаем метаданные
data = scipy.io.loadmat(str(mat_file))

# Посмотрим, что содержится в файле
print("Ключи в файле метаданных:")
for key in data.keys():
    if not key.startswith('__'):
        print(f"- {key}: {type(data[key])}")

# Изучим основные данные
imdb_data = data['imdb'][0, 0]
print(f"\nСтруктура данных IMDB:")
for field_name in imdb_data.dtype.names:
    field_data = imdb_data[field_name]
    print(f"- {field_name}: {field_data.shape if hasattr(field_data, 'shape') else type(field_data)}")

Ключи в файле метаданных:
- imdb: <class 'numpy.ndarray'>

Структура данных IMDB:
- dob: (1, 460723)
- photo_taken: (1, 460723)
- full_path: (1, 460723)
- gender: (1, 460723)
- name: (1, 460723)
- face_location: (1, 460723)
- face_score: (1, 460723)
- second_face_score: (1, 460723)
- celeb_names: (1, 20284)
- celeb_id: (1, 460723)


In [ ]:
import pandas as pd

# Извлекаем данные из структуры
imdb_data = data['imdb'][0, 0]

# Получаем основные поля
full_paths = [path[0] for path in imdb_data['full_path'][0]]
celeb_ids = imdb_data['celeb_id'][0]
celeb_names_raw = imdb_data['celeb_names'][0]

# Создаем словарь ID -> имя знаменитости
celeb_id_to_name = {}
for i, name_array in enumerate(celeb_names_raw):
    if len(name_array) > 0 and len(name_array[0]) > 0:
        celeb_id_to_name[i] = name_array[0]

print(f"Найдено {len(celeb_id_to_name)} уникальных знаменитостей")
print(f"Всего изображений: {len(full_paths)}")

# Создаем датафрейм
dataset_data = []
for i, (path, celeb_id) in enumerate(zip(full_paths, celeb_ids)):
    if celeb_id < len(celeb_names_raw):
        name = celeb_id_to_name.get(celeb_id, "Unknown")
        if name != "Unknown":
            dataset_data.append({
                'celebrity_name': name,
                'image_path': path,
                'celeb_id': celeb_id
            })

df = pd.DataFrame(dataset_data)
print(f"\n Создан датасет с {len(df)} записями")
print("\nПример данных:")
print(df.head())

Найдено 20284 уникальных знаменитостей
Всего изображений: 460723

✅ Создан датасет с 460702 записями

Пример данных:
  celebrity_name                                    image_path  celeb_id
0      Fred Beir   01/nm0000001_rm124825600_1899-5-10_1968.jpg      6488
1      Fred Beir  01/nm0000001_rm3343756032_1899-5-10_1970.jpg      6488
2      Fred Beir   01/nm0000001_rm577153792_1899-5-10_1968.jpg      6488
3      Fred Beir   01/nm0000001_rm946909184_1899-5-10_1968.jpg      6488
4      Fred Beir   01/nm0000001_rm980463616_1899-5-10_1968.jpg      6488


In [ ]:
# Посмотрим на распределение по знаменитостям
celebrity_counts = df['celebrity_name'].value_counts()
print(f"\nТоп-10 знаменитостей по количеству фото:")
print(celebrity_counts.head(10))

print(f"\nОбщее количество уникальных знаменитостей: {len(celebrity_counts)}")


Топ-10 знаменитостей по количеству фото:
celebrity_name
Jennifer Aspen      827
Brad Raider         818
Angeline Ball       795
Courtland Mead      787
Tom Cullen          779
Robert Drivas       739
Nicole Lyn          719
Neil Rayment        719
Zoran Cvijanovic    703
Jim Piddock         696
Name: count, dtype: int64

Общее количество уникальных знаменитостей: 20283


In [ ]:
# URL для загрузки изображений (это большие архивы!)
imdb_images_urls = [
    "https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_0.tar",
    "https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_1.tar",
]

# Сначала загрузим только один архив для тестирования
test_url = imdb_images_urls[0]
test_filename = dataset_dir / "imdb_0.tar"

print("⚠ ВНИМАНИЕ: Архив очень большой (~7GB)")
print("Загружаем первый архив для тестирования...")

# Проверим, не загружен ли уже
if not test_filename.exists():
    download_with_progress(test_url, test_filename)
else:
    print(" Архив уже загружен!")

⚠️ ВНИМАНИЕ: Архив очень большой (~7GB)
Загружаем первый архив для тестирования...
Загружаем /content/imdb_wiki/imdb_0.tar...
Загружено: 100%
✅ /content/imdb_wiki/imdb_0.tar загружен!


In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

# Извлекаем первый архив
print("Извлекаем изображения...")
with tarfile.open(test_filename, 'r') as tar:
    # Извлекаем только первые 100 файлов для тестирования
    members = tar.getmembers()[:100]
    tar.extractall(dataset_dir, members=members)

print(f" Извлечено {len(members)} файлов для тестирования")

# Найдем извлеченные изображения
image_files = list(dataset_dir.rglob("*.jpg"))
print(f"Найдено {len(image_files)} изображений")

if image_files:
    print(f"Пример пути к изображению: {image_files[0]}")

Извлекаем изображения...


/tmp/ipython-input-906066002.py:10: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(dataset_dir, members=members)


✅ Извлечено 100 файлов для тестирования
Найдено 99 изображений
Пример пути к изображению: /content/imdb_wiki/imdb/00/nm0189200_rm925416704_1986-5-28_2011.jpg


In [ ]:
# Работаем напрямую с исходным датафреймом df
celebrity_counts = df['celebrity_name'].value_counts()

# Берем топ-300 знаменитостей (или сколько доступно)
top_300_celebrities = celebrity_counts.head(300).index.tolist()

print(f" Выбрано {len(top_300_celebrities)} знаменитостей")
print(f"Примеры:")
for i, celebrity in enumerate(top_300_celebrities[:5]):
    count = celebrity_counts[celebrity]
    print(f"{i+1}. {celebrity}: {count} фото")

✅ Выбрано 300 знаменитостей
Примеры:
1. Jennifer Aspen: 827 фото
2. Brad Raider: 818 фото
3. Angeline Ball: 795 фото
4. Courtland Mead: 787 фото
5. Tom Cullen: 779 фото


In [ ]:
# Создаем структурированный датасет
structured_data = []

for celebrity in top_300_celebrities:
    # Берем все изображения этой знаменитости
    celebrity_images = df[df['celebrity_name'] == celebrity].head(100)  # Максимум 100

    for idx, (_, row) in enumerate(celebrity_images.iterrows()):
        structured_data.append({
            'celebrity_name': celebrity,
            'image_filename': f"img{idx+1:03d}.jpg",  # img001.jpg, img002.jpg, etc.
            'original_path': row['image_path'],
            'celebrity_id': row['celeb_id']
        })

# Создаем финальный датафрейм
final_df = pd.DataFrame(structured_data)

print(f" Создан структурированный датасет:")
print(f"- Знаменитостей: {final_df['celebrity_name'].nunique()}")
print(f"- Всего изображений: {len(final_df)}")
print(f"- Среднее изображений на знаменитость: {len(final_df) / final_df['celebrity_name'].nunique():.1f}")

# Показываем структуру
print(f"\nПример структуры:")
print(final_df.head())

✅ Создан структурированный датасет:
- Знаменитостей: 300
- Всего изображений: 30000
- Среднее изображений на знаменитость: 100.0

Пример структуры:
   celebrity_name image_filename  \
0  Jennifer Aspen     img001.jpg   
1  Jennifer Aspen     img002.jpg   
2  Jennifer Aspen     img003.jpg   
3  Jennifer Aspen     img004.jpg   
4  Jennifer Aspen     img005.jpg   

                                  original_path  celebrity_id  
0  98/nm0000098_rm1002805248_1969-2-11_1999.jpg          8811  
1  98/nm0000098_rm1006296320_1969-2-11_1994.jpg          8811  
2  98/nm0000098_rm1007481344_1969-2-11_2015.jpg          8811  
3  98/nm0000098_rm1011322624_1969-2-11_2010.jpg          8811  
4  98/nm0000098_rm1016236288_1969-2-11_2009.jpg          8811  


In [ ]:
# Сохраняем CSV с информацией о датасете
final_df.to_csv(dataset_dir / 'structured_dataset_info.csv', index=False)

# Показываем статистику по знаменитостям
celebrity_image_counts = final_df.groupby('celebrity_name').size().sort_values(ascending=False)
print(f"\nСтатистика по изображениям:")
print(f"Максимум изображений у знаменитости: {celebrity_image_counts.max()}")
print(f"Минимум изображений у знаменитости: {celebrity_image_counts.min()}")

print(f"\nТоп-10 знаменитостей в финальном датасете:")
print(celebrity_image_counts.head(10))


Статистика по изображениям:
Максимум изображений у знаменитости: 100
Минимум изображений у знаменитости: 100

Топ-10 знаменитостей в финальном датасете:
celebrity_name
Zoran Cvijanovic    100
Aaron Pearl         100
Adam Saunders       100
Adrienne Bailon     100
Al Roker            100
Alec Guinness       100
Alyson Reed         100
Amanda Steele       100
America Olivo       100
Amy Allen           100
dtype: int64


In [ ]:
# Пусть архив называется imdb_0.tar и лежит в dataset_dir
archive_path = dataset_dir / "imdb_0.tar"

if archive_path.exists():
    print(f"Извлекаем {archive_path.name}...")
    with tarfile.open(archive_path, 'r') as tar:
        tar.extractall(dataset_dir)

    print("Удаляем архив после извлечения...")
    os.remove(archive_path)
    print(" Готово")
else:
    print(" Архив не найден")

Извлекаем imdb_0.tar...


/tmp/ipython-input-4103054443.py:7: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(dataset_dir)


Удаляем архив после извлечения...
✅ Готово


In [ ]:
import os
import pandas as pd
from pathlib import Path
import scipy.io

# Пути к данным
dataset_dir = Path('/content/imdb_wiki')
imdb_folder = dataset_dir / 'imdb'

# Загружаем метаданные
mat_file = dataset_dir / 'imdb' / 'imdb.mat'
data = scipy.io.loadmat(str(mat_file))
imdb_data = data['imdb'][0, 0]

print("Проверяем текущую структуру...")
print(f"Папка с изображениями: {imdb_folder}")
print(f"Файл метаданных: {mat_file}")

# Подсчитываем изображения
if imdb_folder.exists():
    image_files = list(imdb_folder.rglob("*.jpg"))
    print(f"Найдено изображений: {len(image_files)}")

    # Показываем структуру подпапок
    subdirs = [d for d in imdb_folder.iterdir() if d.is_dir()]
    print(f"Подпапки: {[d.name for d in subdirs]}")

    # Примеры файлов
    print("\nПримеры путей к изображениям:")
    for i, img_path in enumerate(image_files[:5]):
        relative_path = img_path.relative_to(dataset_dir)
        print(f"  {i+1}. {relative_path}")
else:
    print(" Папка с изображениями не найдена!")

# Извлекаем метаданные
full_paths = [path[0] for path in imdb_data['full_path'][0]]
celeb_ids = imdb_data['celeb_id'][0]
celeb_names_raw = imdb_data['celeb_names'][0]

print(f"\nМетаданные:")
print(f"  Записей в метаданных: {len(full_paths)}")
print(f"  Уникальных знаменитостей: {len(celeb_names_raw)}")

# Создаем словарь ID -> имя знаменитости
celeb_id_to_name = {}
for i, name_array in enumerate(celeb_names_raw):
    if len(name_array) > 0 and len(name_array[0]) > 0:
        celeb_id_to_name[i] = name_array[0]

print(f"  Знаменитостей с именами: {len(celeb_id_to_name)}")
print(f"\nПримеры знаменитостей:")
for i, (id, name) in enumerate(list(celeb_id_to_name.items())[:5]):
    print(f"  {i+1}. ID {id}: {name}")

🔍 Проверяем текущую структуру...
📁 Папка с изображениями: /content/imdb_wiki/imdb
📄 Файл метаданных: /content/imdb_wiki/imdb/imdb.mat
📸 Найдено изображений: 47578
📂 Подпапки: ['00', '11', '15', '12', '13', '18', '14', '16', '19', '17', '10']

📋 Примеры путей к изображениям:
  1. imdb/00/nm1835900_rm382635520_1978-6-27_2009.jpg
  2. imdb/00/nm0500200_rm1041414144_1982-4-10_2005.jpg
  3. imdb/00/nm0000100_rm952342784_1955-1-6_2003.jpg
  4. imdb/00/nm0063800_rm3529372160_1978-8-20_2015.jpg
  5. imdb/00/nm2003700_rm3503655424_1985-7-18_2007.jpg

📊 Метаданные:
  Записей в метаданных: 460723
  Уникальных знаменитостей: 20284
  Знаменитостей с именами: 20284

🎭 Примеры знаменитостей:
  1. ID 0: 'Lee' George Quinones
  2. ID 1: 'Weird Al' Yankovic
  3. ID 2: 2 Chainz
  4. ID 3: 50 Cent
  5. ID 4: A Martinez


In [ ]:
import shutil
import re
from collections import defaultdict

# Подготавливаем метаданные для сопоставления
print("Подготавливаем метаданные...")

# Создаем словарь: путь_к_файлу -> имя_знаменитости
metadata_mapping = {}

for i, (path, celeb_id) in enumerate(zip(full_paths, celeb_ids)):
    if celeb_id < len(celeb_names_raw):
        name = celeb_id_to_name.get(celeb_id, None)
        if name:
            metadata_mapping[path] = name

print(f"📋 Подготовлено записей метаданных: {len(metadata_mapping)}")

# Показываем примеры
print("\nПримеры сопоставления:")
for i, (path, name) in enumerate(list(metadata_mapping.items())[:5]):
    print(f"  {i+1}. {path} -> {name}")

🔄 Подготавливаем метаданные...
📋 Подготовлено записей метаданных: 460702

📝 Примеры сопоставления:
  1. 01/nm0000001_rm124825600_1899-5-10_1968.jpg -> Fred Beir
  2. 01/nm0000001_rm3343756032_1899-5-10_1970.jpg -> Fred Beir
  3. 01/nm0000001_rm577153792_1899-5-10_1968.jpg -> Fred Beir
  4. 01/nm0000001_rm946909184_1899-5-10_1968.jpg -> Fred Beir
  5. 01/nm0000001_rm980463616_1899-5-10_1968.jpg -> Fred Beir


In [ ]:
import os
import pandas as pd
from pathlib import Path
import scipy.io
import shutil
import re
from collections import defaultdict

# Пути к данным
dataset_dir = Path('/content/imdb_wiki')
imdb_folder = dataset_dir / 'imdb'
mat_file = dataset_dir / 'imdb' / 'imdb.mat'

def clean_filename(name):
    """Очищает имя для использования в качестве имени папки"""
    name = re.sub(r'[<>:"/\\|?*]', '', name)
    name = name.replace(' ', '_')
    return name[:50]

def organize_celebrity_images_fixed(source_dir, output_dir, metadata_dict, min_images=5):
    """Исправленная версия функции организации изображений по знаменитостям"""

    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)

    celebrity_images = defaultdict(list)

    print("Сопоставляем изображения с метаданными...")

    source_images = list(Path(source_dir).rglob("*.jpg"))
    matched_count = 0

    # Создаем обратный индекс метаданных по имени файла
    metadata_by_filename = {}
    for path, celebrity in metadata_dict.items():
        filename = Path(path).name
        if filename not in metadata_by_filename:
            metadata_by_filename[filename] = []
        metadata_by_filename[filename].append(celebrity)

    print(f"Метаданных по именам файлов: {len(metadata_by_filename)}")

    for img_path in source_images:
        filename = img_path.name

        # Пробуем сопоставить по имени файла
        if filename in metadata_by_filename:
            celebrity_name = metadata_by_filename[filename][0]
            celebrity_images[celebrity_name].append(img_path)
            matched_count += 1

    print(f"Сопоставлено: {matched_count} из {len(source_images)} изображений")
    print(f"Найдено знаменитостей: {len(celebrity_images)}")

    # Статистика по количеству изображений
    image_counts = [(name, len(paths)) for name, paths in celebrity_images.items()]
    image_counts.sort(key=lambda x: x[1], reverse=True)

    print(f"\nТоп-10 знаменитостей по количеству изображений:")
    for i, (name, count) in enumerate(image_counts[:10]):
        print(f"  {i+1}. {name}: {count}")

    # Создаем папки
    created_folders = 0
    total_copied = 0

    print(f"\nСоздаем папки (минимум {min_images} изображений)...")

    for celebrity_name, image_paths in celebrity_images.items():
        if len(image_paths) >= min_images:
            folder_name = clean_filename(celebrity_name)
            celebrity_folder = output_dir / folder_name
            celebrity_folder.mkdir(exist_ok=True)

            for i, img_path in enumerate(image_paths):
                new_filename = f"img_{i+1:03d}.jpg"
                dest_path = celebrity_folder / new_filename

                try:
                    shutil.copy2(img_path, dest_path)
                    total_copied += 1
                except Exception as e:
                    print(f"Ошибка копирования {img_path}: {e}")

            created_folders += 1
            print(f"  {folder_name}: {len(image_paths)} изображений")

    print(f"\nГотово!")
    print(f"Создано папок: {created_folders}")
    print(f"Скопировано изображений: {total_copied}")
    print(f"Знаменитостей с недостаточным количеством фото: {len(celebrity_images) - created_folders}")

    return created_folders, total_copied, celebrity_images

def create_dataset_info(celebrity_images, output_dir, min_images=5):
    """Создает CSV файл с информацией о датасете"""

    dataset_info = []

    for celebrity_name, image_paths in celebrity_images.items():
        if len(image_paths) >= min_images:
            folder_name = clean_filename(celebrity_name)

            for i, img_path in enumerate(image_paths):
                dataset_info.append({
                    'celebrity_name': celebrity_name,
                    'folder_name': folder_name,
                    'image_filename': f"img_{i+1:03d}.jpg",
                    'original_path': str(img_path),
                    'image_number': i+1
                })

    df = pd.DataFrame(dataset_info)

    # Сохраняем информацию о датасете
    info_file = Path(output_dir) / 'dataset_info.csv'
    df.to_csv(info_file, index=False)

    print(f"\nИнформация о датасете сохранена в: {info_file}")
    print(f"Записей в датасете: {len(df)}")
    print(f"Уникальных знаменитостей: {df['celebrity_name'].nunique()}")

    return df

# === ОСНОВНОЙ КОД СОЗДАНИЯ ДАТАСЕТА ===

print("=== СОЗДАНИЕ ДАТАСЕТА ЗНАМЕНИТОСТЕЙ ===\n")

# Шаг 1: Проверяем наличие необходимых файлов
print("Шаг 1: Проверка файлов...")
if not imdb_folder.exists():
    print("ОШИБКА: Папка с изображениями не найдена!")
    print("Убедитесь, что архивы IMDB извлечены в /content/imdb_wiki/imdb/")
    exit()

if not mat_file.exists():
    print("ОШИБКА: Файл метаданных не найден!")
    exit()

print(" Все необходимые файлы найдены")

# Шаг 2: Загружаем метаданные
print("\nШаг 2: Загрузка метаданных...")
data = scipy.io.loadmat(str(mat_file))
imdb_data = data['imdb'][0, 0]

# Извлекаем данные
full_paths = [path[0] for path in imdb_data['full_path'][0]]
celeb_ids = imdb_data['celeb_id'][0]
celeb_names_raw = imdb_data['celeb_names'][0]

# Создаем словарь ID -> имя знаменитости
celeb_id_to_name = {}
for i, name_array in enumerate(celeb_names_raw):
    if len(name_array) > 0 and len(name_array[0]) > 0:
        celeb_id_to_name[i] = name_array[0]

# Создаем словарь: путь_к_файлу -> имя_знаменитости
metadata_mapping = {}
for i, (path, celeb_id) in enumerate(zip(full_paths, celeb_ids)):
    if celeb_id < len(celeb_names_raw):
        name = celeb_id_to_name.get(celeb_id, None)
        if name:
            metadata_mapping[path] = name

print(f" Загружено записей метаданных: {len(metadata_mapping)}")
print(f"   Уникальных знаменитостей: {len(celeb_id_to_name)}")

# Шаг 3: Подсчитываем доступные изображения
print("\nШаг 3: Анализ доступных изображений...")
available_images = list(imdb_folder.rglob("*.jpg"))
print(f"   Найдено изображений: {len(available_images)}")

# Шаг 4: Создаем организованный датасет
print("\nШаг 4: Создание датасета...")
output_directory = dataset_dir / 'celebrity_dataset'

# Настройки датасета
MIN_IMAGES_PER_CELEBRITY = 10  # Минимум изображений на знаменитость

folders_created, images_copied, celebrity_data = organize_celebrity_images_fixed(
    source_dir=imdb_folder,
    output_dir=output_directory,
    metadata_dict=metadata_mapping,
    min_images=MIN_IMAGES_PER_CELEBRITY
)

# Шаг 5: Создаем файл с информацией о датасете
print("\nШаг 5: Создание файла с информацией...")
dataset_df = create_dataset_info(
    celebrity_images=celebrity_data,
    output_dir=output_directory,
    min_images=MIN_IMAGES_PER_CELEBRITY
)

# Шаг 6: Финальная статистика
print("\nИТОГОВАЯ СТАТИСТИКА")
print(f"Создано папок со знаменитостями: {folders_created}")
print(f"Скопировано изображений: {images_copied}")
print(f" Средне изображений на знаменитость: {images_copied/folders_created:.1f}")
print(f"Размер датасета: ~{images_copied * 0.5:.0f} MB (примерно)")

# Дополнительная статистика
if len(dataset_df) > 0:
    celebrity_stats = dataset_df.groupby('celebrity_name').size().describe()
    print(f"\nСтатистика по изображениям:")
    print(f"   Минимум: {celebrity_stats['min']:.0f}")
    print(f"   Максимум: {celebrity_stats['max']:.0f}")
    print(f"   Среднее: {celebrity_stats['mean']:.1f}")
    print(f"   Медиана: {celebrity_stats['50%']:.0f}")

print(f"\n Датасет готов! Путь: {output_directory}")
print(f"Информация сохранена в: {output_directory}/dataset_info.csv")

=== СОЗДАНИЕ ДАТАСЕТА ЗНАМЕНИТОСТЕЙ ===

Шаг 1: Проверка файлов...
✅ Все необходимые файлы найдены

Шаг 2: Загрузка метаданных...
✅ Загружено записей метаданных: 460702
   Уникальных знаменитостей: 20284

Шаг 3: Анализ доступных изображений...
   Найдено изображений: 46641

Шаг 4: Создание датасета...
Сопоставляем изображения с метаданными...
Метаданных по именам файлов: 460702
Сопоставлено: 46641 из 46641 изображений
Найдено знаменитостей: 2088

Топ-10 знаменитостей по количеству изображений:
  1. Angeline Ball: 795
  2. Drew Carey: 532
  3. Reeve Carney: 514
  4. Hugh M. Hefner: 473
  5. Kali Rocha: 432
  6. Naomie Harris: 414
  7. Bryan Cuprill: 398
  8. Kevin Bishop: 386
  9. Keegan Allen: 383
  10. Pippa Bennett-Warner: 375

Создаем папки (минимум 10 изображений)...
  Milauna_Jackson: 223 изображений
  Brendan_Gleeson: 232 изображений
  Richard_Carlson: 53 изображений
  Martin_Garner: 139 изображений
  Ayesha_Dharker: 37 изображений
  Kelly_Reno: 62 изображений
  Daniel_Buran: 71 

In [ ]:
import os
import shutil
from pathlib import Path

def cleanup_space():
    """
    Удаляет все временные файлы, оставляя только готовый датасет celebrity_dataset
    """

    # Путь к основной папке
    dataset_dir = Path('/content/imdb_wiki')

    # Путь к готовому датасету (этот сохраняем!)
    final_dataset = dataset_dir / 'celebrity_dataset'

    print("🗑 Начинаем очистку пространства...")

    # Проверяем, что готовый датасет существует
    if not final_dataset.exists():
        print(" ОШИБКА: Готовый датасет не найден! Остановка.")
        return

    print(f" Готовый датасет найден: {final_dataset}")

    freed_space = 0

    # Удаляем архивы
    archives_to_remove = [
        'imdb_0.tar',
        'imdb_1.tar',
        'imdb_2.tar',
        'imdb_meta.tar'
    ]

    for archive_name in archives_to_remove:
        archive_path = dataset_dir / archive_name
        if archive_path.exists():
            size_mb = archive_path.stat().st_size / (1024 * 1024)
            archive_path.unlink()
            freed_space += size_mb
            print(f"🗑 Удален архив: {archive_name} ({size_mb:.1f} MB)")

    # Удаляем извлеченные изображения (папка imdb)
    imdb_folder = dataset_dir / 'imdb'
    if imdb_folder.exists():
        # Подсчитываем размер папки
        total_size = 0
        for root, dirs, files in os.walk(imdb_folder):
            for file in files:
                try:
                    file_path = os.path.join(root, file)
                    total_size += os.path.getsize(file_path)
                except (OSError, FileNotFoundError):
                    pass

        size_mb = total_size / (1024 * 1024)

        # Удаляем папку
        shutil.rmtree(imdb_folder)
        freed_space += size_mb
        print(f"Удалена папка с изображениями: imdb/ ({size_mb:.1f} MB)")

    # Удаляем CSV файл с информацией
    csv_file = dataset_dir / 'structured_dataset_info.csv'
    if csv_file.exists():
        size_mb = csv_file.stat().st_size / (1024 * 1024)
        csv_file.unlink()
        freed_space += size_mb
        print(f"Удален файл: structured_dataset_info.csv ({size_mb:.3f} MB)")

    print(f"\n ОЧИСТКА ЗАВЕРШЕНА!")
    print(f"Освобождено места: {freed_space:.1f} MB (~{freed_space/1024:.1f} GB)")

    # Проверяем, что осталось
    remaining_items = list(dataset_dir.iterdir())
    print(f"\n📁 Что осталось в /content/imdb_wiki/:")
    for item in remaining_items:
        if item.is_dir():
            # Подсчитываем размер папки
            total_size = 0
            file_count = 0
            for root, dirs, files in os.walk(item):
                file_count += len(files)
                for file in files:
                    try:
                        file_path = os.path.join(root, file)
                        total_size += os.path.getsize(file_path)
                    except (OSError, FileNotFoundError):
                        pass
            size_mb = total_size / (1024 * 1024)
            print(f"{item.name}/: {file_count} файлов, {size_mb:.1f} MB")
        else:
            size_mb = item.stat().st_size / (1024 * 1024)
            print(f"{item.name}: {size_mb:.3f} MB")

    # Информация о датасете
    dataset_info_file = final_dataset / 'dataset_info.csv'
    if dataset_info_file.exists():
        print(f"\Готовый датасет готов к использованию!")
        print(f"   Путь: {final_dataset}")
        print(f"   Информация: {dataset_info_file}")

# Запускаем очистку
cleanup_space()

🗑️ Начинаем очистку пространства...
✅ Готовый датасет найден: /content/imdb_wiki/celebrity_dataset
🗑️ Удален архив: imdb_meta.tar (21.9 MB)
🗑️ Удалена папка с изображениями: imdb/ (27366.5 MB)
🗑️ Удален файл: structured_dataset_info.csv (2.151 MB)

🎉 ОЧИСТКА ЗАВЕРШЕНА!
📊 Освобождено места: 27390.6 MB (~26.7 GB)

📁 Что осталось в /content/imdb_wiki/:
  📂 celebrity_dataset/: 42415 файлов, 24057.6 MB

📋 Готовый датасет готов к использованию!
   Путь: /content/imdb_wiki/celebrity_dataset
   Информация: /content/imdb_wiki/celebrity_dataset/dataset_info.csv


In [ ]:
import os
import shutil
from collections import Counter
import zipfile

# исходная папка
base_dir = "/content/imdb_wiki/celebrity_dataset"
output_dir = "/content/top_100_celebrities"

# создаём папку для результата
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

# собираем количество файлов в каждой подпапке
folder_counts = {}
for root, dirs, files in os.walk(base_dir):
    if root == base_dir:  # проверяем только директории верхнего уровня
        for d in dirs:
            folder_path = os.path.join(base_dir, d)
            num_files = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
            folder_counts[d] = num_files

# выбираем топ-100 по числу файлов
top_100 = [folder for folder, _ in Counter(folder_counts).most_common(100)]

# копируем топ-100 папок в новую директорию
for folder in top_100:
    src = os.path.join(base_dir, folder)
    dst = os.path.join(output_dir, folder)
    shutil.copytree(src, dst)

# создаём ZIP-архив
zip_path = "/content/top_100_celebrities.zip"
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', output_dir)

print("Готово! Архив сохранён по пути:", zip_path)

Готово! Архив сохранён по пути: /content/top_100_celebrities.zip


In [ ]:
# Разбиваем zip на части по 2 ГБ
!split -b 2000M /content/top_100_celebrities.zip /content/top_100_celebrities_part_

# Проверим, какие файлы получились
!ls -lh /content/top_100_celebrities_part_*

-rw-r--r-- 1 root root 2.0G Sep 25 15:28 /content/top_100_celebrities_part_aa
-rw-r--r-- 1 root root 2.0G Sep 25 15:28 /content/top_100_celebrities_part_ab
-rw-r--r-- 1 root root 2.0G Sep 25 15:29 /content/top_100_celebrities_part_ac
-rw-r--r-- 1 root root 2.0G Sep 25 15:29 /content/top_100_celebrities_part_ad
-rw-r--r-- 1 root root 2.0G Sep 25 15:30 /content/top_100_celebrities_part_ae
-rw-r--r-- 1 root root 2.0G Sep 25 15:30 /content/top_100_celebrities_part_af
-rw-r--r-- 1 root root 192M Sep 25 15:30 /content/top_100_celebrities_part_ag


In [ ]:
from google.colab import files
files.download("/content/top_100_celebrities_part_aa")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download("/content/top_100_celebrities_part_ab")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download("/content/top_100_celebrities_part_ac")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download("/content/top_100_celebrities_part_ad")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download("/content/top_100_celebrities_part_ae")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download("/content/top_100_celebrities_part_af")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download("/content/top_100_celebrities_part_ag")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>